In [1]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import src.data.breathe_data as bd
import src.data.helpers as dh
import src.inference.helpers as ih
import src.modelling_ar.ar as ar
import src.inf_cutset_conditioning.helpers as cutseth
import src.inf_cutset_conditioning.cutset_cond_algs_learn_ar_change as cca_ar_change
import src.models.helpers as mh

import json
import numpy as np

In [31]:
# Load data
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

INFO:root:* Checking for same day measurements *


In [3]:
id = "134"
dftmp, start_idx, end_idx = dh.find_longest_consec_series(df[df.ID == id], n_days=3)


ecfev1_noise_model_suffix = "_std_add_mult_fev1"
ar_prior = "breathe (2 days model, ecFEV1 addmultnoise, ecFEF25-75)"
ar_change_cpt_suffix = "_shape_factor1"

(
    # p_M_given_D,
    # log_p_D_given_M,
    # AR_given_M_and_D,
    # AR_given_M_and_all_D,
    log_p_S_given_D,
    res_dict,
) = cca_ar_change.run_long_noise_model_through_time(
    dftmp,
    ar_prior=ar_prior,
    ar_change_cpt_suffix=ar_change_cpt_suffix,
    ecfev1_noise_model_suffix=ecfev1_noise_model_suffix,
)

Warning - min_possible_hfev1_under_model: 6
134 - Time for 18 entries: 728.41 s


## Process results

In [147]:
def load_and_process_json(card, params, labels):
    path = f"{dh.get_path_to_src()}inf_cutset_conditioning/p_s_given_d_card{card}.json"

    # Load the JSON file
    with open(path, "r") as file:
        data = json.load(file)

    # Convert lists to NumPy arrays
    for key in data:
        data[key] = np.array(data[key])

    # Replace 'nan' with np.nan and "-inf" with -np.inf
    for key in data:
        data[key][np.where(data[key] == "-inf")] = -np.inf

    # REmove dtypes from data
    data = {key: data[key].astype(float) for key in data}

    # Remove entries with only nan values
    data = {key: data[key] for key in data if not np.all(np.isnan(data[key]))}

    log_p_s_given_D = np.sum(list(data.values()), axis=0)

    df_heatmap = pd.DataFrame(
        columns=["log_p_S_given_D", "params"], data=zip(log_p_s_given_D, params)
    )
    # Split params into three columns "main width", "tail width", "main weight"
    df_heatmap[labels] = pd.DataFrame(
        df_heatmap["params"].tolist(), index=df_heatmap.index
    )

    IDs = data.keys()
    return df_heatmap, IDs

In [148]:
# Element wise addition of the values in data
params = [
    (2, 15, 0.3),
    (2, 15, 0.5),
    (2, 15, 0.7),
    (2, 30, 0.3),
    (2, 30, 0.5),
    (2, 30, 0.7),
    (2, 50, 0.3),
    (2, 50, 0.5),
    (2, 50, 0.7),
    (5, 15, 0.3),
    (5, 15, 0.5),
    (5, 15, 0.7),
    (5, 30, 0.3),
    (5, 30, 0.5),
    (5, 30, 0.7),
    (5, 50, 0.3),
    (5, 50, 0.5),
    (5, 50, 0.7),
    (8, 15, 0.3),
    (8, 15, 0.5),
    (8, 15, 0.7),
    (8, 30, 0.3),
    (8, 30, 0.5),
    (8, 30, 0.7),
    (8, 50, 0.3),
    (8, 50, 0.5),
    (8, 50, 0.7),
]
labels = ["main width", "tail width", "main weight"]

df_heatmap, IDs = load_and_process_json(27, params, labels)
df_heatmap

,log_p_S_given_D,params,main width,tail width,main weight
0,-5.094301e+04,"(2, 15, 0.3)",2,15,0.3
1,-4.910879e+04,"(2, 15, 0.5)",2,15,0.5
2,-4.775033e+04,"(2, 15, 0.7)",2,15,0.7
3,-5.177770e+04,"(2, 30, 0.3)",2,30,0.3
4,-4.944245e+04,"(2, 30, 0.5)",2,30,0.5
5,-4.788089e+04,"(2, 30, 0.7)",2,30,0.7
6,-5.141493e+04,"(2, 50, 0.3)",2,50,0.3
7,-4.909168e+04,"(2, 50, 0.5)",2,50,0.5
8,-4.767621e+04,"(2, 50, 0.7)",2,50,0.7
9,-5.278688e+04,"(5, 15, 0.3)",5,15,0.3


In [135]:
# Plot heatmap with main width on x axis, tail width on y axis and log_p_S_given_D as the color, just use the first 3 main weights

fig = make_subplots(1, 3)


def add_heatmap_for_weight(df, weight, col):
    df = df[df["main weight"] == weight]
    fig.add_trace(
        go.Heatmap(z=df["log_p_S_given_D"], x=df["main width"], y=df["tail width"]),
        row=1,
        col=col,
    )
    fig.update_xaxes(
        type="category",
        row=1,
        col=col,
        title="Main width<br><br>Main weight=" + str(weight),
    )


add_heatmap_for_weight(df_heatmap, 0.3, 1)
add_heatmap_for_weight(df_heatmap, 0.5, 2)
add_heatmap_for_weight(df_heatmap, 0.7, 3)

fig.update_layout(
    title=f"Log probability of S given D for different AR change parameters ({len(IDs.keys())} IDs)",
    yaxis_title="Tail width",
    # yaxis_type='log',
    width=650,
    height=300,
    font_size=10,
)
# Recude font size of the subplot titles
for annotation in fig["layout"]["annotations"]:
    annotation["font"] = dict(size=14)
# use the same color scale for all subplots
fig.update_traces(
    zmin=min(df_heatmap.log_p_S_given_D[df_heatmap.log_p_S_given_D != -np.inf]),
    zmax=max(df_heatmap.log_p_S_given_D),
    colorbar=dict(title="log_p_S_given_D"),
)
# X axis and y axis are categorical
fig.update_yaxes(type="category")

fig.show()

In [ ]:
# The weights is the most differentiating parameter, then main width. Tail width is a smaller effect, althought very important, and remains hidden
# 0.7 weight
# std 2 -> laplace would be better
# tail width doesn't change. Go wider?

# Since the tails is a secondary effet, let's find the rest first

# 3 laplace
# 3 gaussians

In [149]:
params = [
    (0.5, False),
    (0.5, True),
    (1, False),
    (1, True),
    (1.5, False),
    (1.5, True),
    (2, False),
    (2, True),
    (2.5, False),
    (2.5, True),
]
labels = ["main width", "laplace"]

df_10, IDs_10 = load_and_process_json(10, params, labels)
df_10

,log_p_S_given_D,params,main width,laplace
0,-6.655665e+04,"(0.5, False)",0.5,False
1,-6.628876e+04,"(0.5, True)",0.5,True
2,-6.761041e+04,"(1, False)",1.0,False
3,-6.730061e+04,"(1, True)",1.0,True
4,-6.848776e+04,"(1.5, False)",1.5,False
5,-6.834117e+04,"(1.5, True)",1.5,True
6,-6.921651e+04,"(2, False)",2.0,False
7,-6.927346e+04,"(2, True)",2.0,True
8,-inf,"(2.5, False)",2.5,False
9,-inf,"(2.5, True)",2.5,True


In [164]:
# Plot heatmap with main width on x axis, tail width on y axis and log_p_S_given_D as the color, just use the first 3 main weights

tail_width = 30
main_weight = 0.7

fig = make_subplots(1, 1)

fig.add_trace(
    go.Heatmap(z=df_10["log_p_S_given_D"], x=df_10["main width"], y=df_10["laplace"]),
    row=1,
    col=1,
)
fig.update_xaxes(
    type="category",
    row=1,
    col=1,
    title="Main width",
)

fig.update_layout(
    title=f"Log P(S|D) ({len(IDs_10)} IDs)<br>tail width={tail_width}, main weight={main_weight}",
    yaxis_title="Laplace",
    # yaxis_type='log',
    width=400,
    height=250,
    font_size=10,
)
# Recude font size of the subplot titles
for annotation in fig["layout"]["annotations"]:
    annotation["font"] = dict(size=14)
# use the same color scale for all subplots
fig.update_traces(
    zmin=min(df_10.log_p_S_given_D[df_10.log_p_S_given_D != -np.inf]),
    zmax=max(df_10.log_p_S_given_D),
    colorbar=dict(title="log_p_S_given_D"),
)
# X axis and y axis are categorical
fig.update_yaxes(type="category")

fig.show()